In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.float_format', '{:.2f}'.format)

In [2]:
from sklearn import model_selection, preprocessing

In [3]:
# Загружаем данные
data = open('corpus').read()
labels, texts = [], []
for i, line in enumerate(data.split("\n")):
    content = line.split()
    labels.append(content[0])
    texts.append(" ".join(content[1:]))

# создаем df
trainDF = pd.DataFrame()
trainDF['text'] = texts
trainDF['label'] = labels
encoder = preprocessing.LabelEncoder()
trainDF['label'] = encoder.fit_transform(trainDF['label'])
trainDF.head(5)

,text,label
0,Stuning even for the non-gamer: This sound tra...,1
1,The best soundtrack ever to anything.: I'm rea...,1
2,Amazing!: This soundtrack is my favorite music...,1
3,Excellent Soundtrack: I truly like this soundt...,1
4,"Remember, Pull Your Jaw Off The Floor After He...",1


In [4]:
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(trainDF['text'], trainDF['label'])

--------

In [5]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, Conv1D, GRU, LSTM, Dropout
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.optimizers import Adam

In [6]:
train_data = tf.data.Dataset.from_tensor_slices((train_x, train_y))
valid_data = tf.data.Dataset.from_tensor_slices((valid_x, valid_y))
train_data = train_data.batch(16)
valid_data = valid_data.batch(16)

In [7]:
# for raw in train_data.take(1):
#     print(raw)

In [8]:
AUTOTUNE = tf.data.AUTOTUNE
train_data = train_data.cache().prefetch(buffer_size=AUTOTUNE)
valid_data = valid_data.cache().prefetch(buffer_size=AUTOTUNE)

In [9]:
def custom_standardization(input_data):
    
    return input_data

# ***************************************************************************
vocab_size = 10000
seq_len = 100

vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=seq_len)

# Make a text-only dataset (no labels) and call adapt to build the vocabulary.
text_data = train_data.map(lambda x, y: x)
vectorize_layer.adapt(text_data)

In [10]:
embedding_dim=200

model = Sequential([
    vectorize_layer,
    Embedding(vocab_size, embedding_dim, name="embedding"),
    Conv1D(20, (3), activation='relu'),
    Conv1D(30, (2), activation='relu'),
    Conv1D(50, (2), activation='relu'),
    GRU(100),
    #GlobalAveragePooling1D(),
    Dense(100, activation='relu'),
    Dense(1, activation='sigmoid')
])

In [11]:
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [12]:
model.fit(train_data, validation_data=valid_data, epochs=15)

Epoch 1/15
469/469 [==============================] - 21s 35ms/step - loss: 0.6929 - accuracy: 0.5121 - val_loss: 0.6933 - val_accuracy: 0.5012
Epoch 2/15
469/469 [==============================] - 15s 32ms/step - loss: 0.6897 - accuracy: 0.5120 - val_loss: 0.6916 - val_accuracy: 0.5168
Epoch 3/15
469/469 [==============================] - 15s 32ms/step - loss: 0.6595 - accuracy: 0.5717 - val_loss: 0.6934 - val_accuracy: 0.5572
Epoch 4/15
469/469 [==============================] - 15s 33ms/step - loss: 0.5677 - accuracy: 0.6535 - val_loss: 0.6663 - val_accuracy: 0.6612
Epoch 5/15
469/469 [==============================] - 15s 33ms/step - loss: 0.3884 - accuracy: 0.8096 - val_loss: 0.5257 - val_accuracy: 0.7936
Epoch 6/15
469/469 [==============================] - 15s 32ms/step - loss: 0.1842 - accuracy: 0.9271 - val_loss: 0.6400 - val_accuracy: 0.7936
Epoch 7/15
469/469 [==============================] - 15s 32ms/step - loss: 0.0927 - accuracy: 0.9703 - val_loss: 0.9020 - val_accuracy:

*********************

In [13]:
class myNet(tf.keras.Model):
    def __init__(self):
        super(myNet, self).__init__()
        self.emb = Embedding(vocab_size, embedding_dim, name="embedding")
        self.conv1 = Conv1D(200, (3))
        self.conv2 = Conv1D(200, (3))
        self.gPool = GlobalAveragePooling1D()
        self.fc1 = Dense(100, activation='relu')
        self.fc2 = Dense(1)
        
        self.ss = Sequential([
            vectorize_layer,
            Embedding(vocab_size, embedding_dim, name="embedding"),
            Conv1D(200, (3)),
            Conv1D(200, (2)),
            GRU(300),
#             GlobalAveragePooling1D(),
            Dense(100, activation='relu'),
            Dense(1)
        ])

    def call(self, x):
        x = vectorize_layer(x)
        x = self.emb(x)
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.gPool(x)
        x = self.fc1(x)
#         x = self.ss(x)
        return self.fc2(x)

In [14]:
mmodel = myNet()

In [15]:
mmodel.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
mmodel.fit(train_data, validation_data=valid_data, epochs=15)